In [1]:
import numpy as np
import cPickle, gzip

from neon import logger as neon_logger
from neon.callbacks.callbacks import Callbacks
from neon.data import MNIST
from neon.initializers import Gaussian, GlorotUniform
from neon.layers import Affine, BranchNode, Multicost, Tree, GeneralizedCost, SingleOutputTree
from neon.models import Model
from neon.optimizers import RMSProp
from neon.transforms import Rectlin, Logistic, Softmax
from neon.transforms import CrossEntropyMulti, Misclassification
from neon.util.argparser import NeonArgparser
from neon.data import ArrayIterator
from neon.backends import gen_backend

In [52]:
from neon.backends import gen_backend
be = gen_backend(backend='cpu',batch_size=10)
train = ArrayIterator(X=[features_train[0], features_train[1]], 
                      y=labels_train[0], nclass=n_out[0])

In [53]:
from neon.data import NervanaDataIterator
import numpy as np
import cPickle
import os

class SVHN(NervanaDataIterator):

    def __init__(self, X, Y, nclass):

        # Load the numpy data into some variables. We divide the image by 255 to normalize the values
        # between 0 and 1.
        self.X = X
        self.Y = Y
        self.nclass_t1 = nclass[0]
        self.nclass_t2 = nclass[1]
        self.nclass_t3 = nclass[2]
#         self.shape = lshape  # shape of the input data (e.g. for images, (C, H, W))

        # 1. assign some required and useful attributes
        self.start = 0  # start at zero
        self.ndata = self.X[0].shape[0]  # number of images in X (hint: use X.shape)
        self.nfeatures = self.X[0].shape[1]  # number of features in X (hint: use X.shape)
        self.Y_t1 = onehot_gen(self.Y[0], self.ndata, self.nclass_t1)
        self.Y_t2 = onehot_gen(self.Y[1], self.ndata, self.nclass_t2)
        self.Y_t3 = onehot_gen(self.Y[2], self.ndata, self.nclass_t3)

        # number of minibatches per epoch
        # to calculate this, use the batchsize, which is stored in self.be.bsz
        self.nbatches = self.ndata/self.be.bsz 
        
        
        # 2. allocate memory on the GPU for a minibatch's worth of data.
        # (e.g. use `self.be` to access the backend.). See the backend documentation.
        # to get the minibatch size, use self.be.bsz
        # hint: X should have shape (# features, mini-batch size)
        # hint: use some of the attributes previously defined above
        self.dev_X_t1 = self.be.zeros((self.nfeatures, self.be.bsz))
        self.dev_X_t2 = self.be.zeros((self.nfeatures, self.be.bsz))
        self.dev_X_t3 = self.be.zeros((self.nfeatures, self.be.bsz))
        self.dev_Y_t1 = self.be.zeros((self.Y[0].shape[1], self.be.bsz))
        self.dev_Y_t2 = self.be.zeros((self.Y[1].shape[1], self.be.bsz))
        self.dev_Y_t3 = self.be.zeros((self.Y[2].shape[1], self.be.bsz))


    def reset(self):
        self.start = 0
        
    def onehot_gen(z, s, n):
        self.b = np.zeros((s, n))
        self.b[np.arange(s), z] = 1
        return b    
    def __iter__(self):
        # 3. loop through minibatches in the dataset
        for index in range(self.start, self.ndata, self.be.bsz):
            # 3a. grab the right slice from the numpy arrays
            inputs_t1 = self.X[0][index:(index + self.be.bsz), :]
            targets_t1 = self.Y_t1[index:(index + self.be.bsz), :]
            inputs_t2 = self.X[1][index:(index + self.be.bsz), :]
            targets_t2 = self.Y_t2[index:(index + self.be.bsz), :]
            inputs_t3 = self.X[2][index:(index + self.be.bsz), :]
            targets_t3 = self.Y_t3[index:(index + self.be.bsz), :]
            
            
            # The arrays X and Y data are in shape (batch_size, num_features),
            # but the iterator needs to return data with shape (num_features, batch_size).
            # here we transpose the data, and then store it as a contiguous array. 
            # numpy arrays need to be contiguous before being loaded onto the GPU.
            inputs_t1 = np.ascontiguousarray(inputs_t1.T)
            targets_t1 = np.ascontiguousarray(targets_t1.T)
            inputs_t2 = np.ascontiguousarray(inputs_t2.T)
            targets_t2 = np.ascontiguousarray(targets_t2.T)
            inputs_t3 = np.ascontiguousarray(inputs_t3.T)
            targets_t3 = np.ascontiguousarray(targets_t3.T)
                        
            # here we test your implementation
            # your slice has to have the same shape as the GPU tensors you allocated
#             assert inputs[0].shape == self.dev_X_t1.shape, \
#                    "inputs has shape {}, but dev_X is {}".format(inputs.shape, self.dev_X.shape)
#             assert targets[0].shape == self.dev_Y_t1.shape, \
#                    "targets has shape {}, but dev_Y is {}".format(targets.shape, self.dev_Y.shape)
            
            # 3b. transfer from numpy arrays to device
            # - use the GPU memory buffers allocated previously,
            #    and call the myTensorBuffer.set() function. 
            self.dev_X_t1.set(inputs_t1)
            self.dev_Y_t1.set(targets_t1)
            self.dev_X_t2.set(inputs_t2)
            self.dev_Y_t2.set(targets_t2)
            self.dev_X_t3.set(inputs_t3)
            self.dev_Y_t3.set(targets_t3)
            
            # 3c. yield a tuple of the device tensors.
            # X should be of shape (num_features, batch_size)
            # Y should be of shape (4, batch_size)
            yield ((self.dev_X_t1, self.dev_X_t2, self.dev_X_t3 ),
                   (self.dev_Y_t1, self.dev_Y_t2, self.dev_Y_t3))

In [54]:
# Set up a number of initial variables for use with baseline
NUM_TASKS = 3; # number of learning tasks (for multi-task learning)
NUM_FOLDS = 10; # number of folds for training (main cross validation loop)
NUM_EPOCH = 5; # number of epochs



truth_a_arr = []
pred_a_arr = []

truth_b_arr = []
pred_b_arr = []

truth_c_arr = []
pred_c_arr = []


In [55]:
for fold in range( NUM_FOLDS ):

    features_train = []
    labels_train = []
    truths_train = []

    features_test = []
    labels_test = []
    truths_test = []

    n_out = []

    for task in range( NUM_TASKS ):
        file_post = '.' + str(task) + '.' + str(fold) + '.pkl.gz'
        fname_train = 'train/train' + file_post; 
        fname_test  = 'test/test' + file_post; 
        with gzip.open( fname_train, 'rb' ) as f:
            feature_train, label_train = cPickle.load( f )

        with gzip.open( fname_test, 'rb') as f:
            feature_test, label_test = cPickle.load( f )

        features_train.append( feature_train )
        labels_train.append( label_train )

        features_test.append( feature_test )
        labels_test.append( label_test )

        mv = np.max( label_train )
        truth_train = np.zeros( ( len( label_train ), mv + 1 ) )
        for i in range( len( label_train ) ):
            truth_train[ i, label_train[ i ] ] = 1

        truths_train.append( truth_train )

        mv = np.max( label_test )
        truth_test = np.zeros( ( len( label_test ), mv + 1 ) )
        for i in range( len( label_test ) ):
            truth_test[ i, label_test[ i ] ] = 1

        truths_test.append( truth_test )

        n_out.append( mv + 1 )

    flen = len( feature_train[ 0 ] ); # input feature length is set to 400 for now based on the training examples available.


In [56]:
for x in features_train:
    print x.shape

(1200, 400)
(1200, 400)
(1200, 400)


In [65]:
print labels_train[0]
a = onehot_gen(labels_train[0], 1200, n_out[0])
print labels_train[0].shape
print a.shape

[0 0 0 ..., 1 1 1]
(1200,)
(1200, 2)


In [63]:
be.zeros((3,3,3))

CPUTensor(base 0x105069030) name:None shape:(3, 3, 3) dtype:float32 strides:(36, 12, 4) is_c_contiguous:True

In [58]:
train = SVHN(features_train, labels_train, n_out)

IndexError: tuple index out of range

In [78]:
train_t1 = ArrayIterator(X=features_train[0], y=labels_train[0], nclass=n_out[0])
train_t2 = ArrayIterator(X=features_train[1], y=labels_train[1], nclass=n_out[1])
train_t3 = ArrayIterator(X=features_train[2], y=labels_train[2], nclass=n_out[2])

test_t1 = ArrayIterator(X=features_test[0], y=labels_test[0], nclass=n_out[0])
test_t2 = ArrayIterator(X=features_test[1], y=labels_test[1], nclass=n_out[1])
test_t3 = ArrayIterator(X=features_test[2], y=labels_test[2], nclass=n_out[2])

In [79]:
b1 = BranchNode(name='b1')
b2 = BranchNode(name='b2')
init = GlorotUniform()

In [96]:
p1 = [Affine(nout=flen, name="t1_input", activation=Rectlin(), init=init), 
      b1,
      Affine(nout=flen, name="share", activation=Rectlin(), init=init),
      b2,
      Affine(nout=flen, name="t1_3", activation=Rectlin(), init=init),
      Affine(nout=256, name="t1_4", activation=Rectlin(), init=init),
      Affine(nout=n_out[0], name="t2_out", activation=Rectlin(), init=init)]
p2 = [Affine(nout=flen, name="t1_input", activation=Rectlin(), init=init), 
      b1,
      Affine(nout=flen, name="share", activation=Rectlin(), init=init),
      b2, 
      Affine(nout=flen, name="t2_3", activation=Rectlin(), init=init),
      Affine(nout=256, name="t2_4", activation=Rectlin(), init=init),
      Affine(nout=n_out[1], name="t2_out", activation=Rectlin(), init=init)]

In [109]:
model = Model(layers=SingleOutputTree([p1, p2]))


In [110]:
optimizer = RMSProp(learning_rate=0.001)
cost = Multicost(costs=[GeneralizedCost(costfunc=CrossEntropyMulti(usebits=True))])
                        
callbacks_1 = Callbacks(model_1, eval_set=test_t1, eval_freq=NUM_EPOCH)
callbacks_2 = Callbacks(model_2, eval_set=test_t2, eval_freq=NUM_EPOCH)
callbacks_3 = Callbacks(model_3, eval_set=test_t3, eval_freq=NUM_EPOCH)

callbacks_1 = Callbacks(model_1, eval_set=test_t1, eval_freq=NUM_EPOCH)

In [111]:
cost1 = CrossEntropyMulti(usebits=True)

In [112]:
# model_1.fit(train_t1, optimizer=optimizer, cost=cost, callbacks=callbacks_1, 
#             num_epochs=1)
# model_2.fit(train_t2, optimizer=optimizer, cost=cost, 
#             callbacks=callbacks_2, num_epochs=NUM_EPOCH)
# model_3.fit(train_t3, optimizer=optimizer, cost=cost, 
#             callbacks=callbacks_3, num_epochs=NUM_EPOCH)
model_4.fit(train_t1, optimizer=optimizer, cost=cost, callbacks=callbacks_1, 
            num_epochs=NUM_EPOCH)

TypeError: an integer is required

In [41]:
pdict = model_1.get_description(get_weights=True)

In [43]:
pdict2 = model_2.get_description(get_weights=True)
pdict3 = model_3.get_description(get_weights=True)

In [34]:
def onehot_gen(z, s, n):
    b = np.zeros((s, n))
    b[np.arange(s), z] = 1
    return b

In [35]:
a = [0,1,1,3,2]
b = onehot_gen(a, len(a), 4)
print b

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  0.]]
